<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-model" data-toc-modified-id="load-model-1">load model</a></span></li><li><span><a href="#test-model" data-toc-modified-id="test-model-2">test model</a></span></li></ul></div>

# load model

In [1]:
import os
import glob
import cv2
import tensorflow as tf
import numpy as np
import json
from tqdm.notebook import tqdm
import collections

In [2]:
work_dir = os.path.join(os.getenv("HOME"), "UDIGO")
model_dir = os.path.join(work_dir, "models")
test_dir = os.path.join(work_dir, "test_data")
data_dir = os.path.join(work_dir, "data")

In [3]:
model_list = os.listdir(model_dir)
model_list

['test5_efnb0_24-0.01-1.00.h5',
 'place23_efnb0_3-0.26-0.92.h5',
 'test5_efnb0_28-0.01-1.00.h5',
 'place10_efnb0_9-0.04-0.99.h5',
 'place10_efnb1_6-0.02-0.99.h5']

In [4]:
model = tf.keras.models.load_model(os.path.join(model_dir, model_list[1]))

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 23)                11799     
Total params: 4,719,290
Trainable params: 4,676,243
Non-trainable params: 43,047
_________________________________________________________________


# test model

In [6]:
def test_model(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = image[np.newaxis, :, :, :]
    pred = model.predict(image, batch_size=1)
    return np.argmax(pred)

In [7]:
with open(os.path.join(data_dir, "place_23_label.json"), "r") as f:
    label_dict = json.load(f)

In [8]:
image_list = glob.glob(test_dir + "/*/*")
len(image_list)

4518

In [9]:
image_list[:10]

['/home/ssac21/UDIGO/test_data/산/87.jpg',
 '/home/ssac21/UDIGO/test_data/산/183.jpg',
 '/home/ssac21/UDIGO/test_data/산/138.jpg',
 '/home/ssac21/UDIGO/test_data/산/6.jpg',
 '/home/ssac21/UDIGO/test_data/산/178.jpg',
 '/home/ssac21/UDIGO/test_data/산/180.jpg',
 '/home/ssac21/UDIGO/test_data/산/67.jpg',
 '/home/ssac21/UDIGO/test_data/산/192.jpg',
 '/home/ssac21/UDIGO/test_data/산/184.jpg',
 '/home/ssac21/UDIGO/test_data/산/153.jpg']

In [10]:
label_list = []
correct_dict = {}
correct = 0
n = 0

for image in tqdm(image_list):
    try:
        pred = test_model(image)
        n += 1

        label = image.split("/")[-2]
        label_list.append(label)
        if label not in correct_dict:
            correct_dict[label] = 0

        label_encoding = label_dict[label]
        if pred == label_encoding:
            correct_dict[label] += 1
            correct += 1
    
    except:
        print("Invalid image:", image)
        continue
        
print(f"{n} data, test complete!")

Invalid image: /home/ssac21/UDIGO/test_data/수영장/c02_01_04_img2.gif
Invalid image: /home/ssac21/UDIGO/test_data/수영장/c02_04_04_img1.gif
Invalid image: /home/ssac21/UDIGO/test_data/수영장/facil_intro02.gif
Invalid image: /home/ssac21/UDIGO/test_data/수영장/1513145177337_36103fc5a6d9429c90c4d9d033df365e.gif
Invalid image: /home/ssac21/UDIGO/test_data/폭포/8c214fd58507879d1c2e7ab5cca31743.gif
Invalid image: /home/ssac21/UDIGO/test_data/아쿠아리움/070506553201908240084b19e-97ec-4ee2-b208-2c01c3c3fa1d.gif
Invalid image: /home/ssac21/UDIGO/test_data/공원/30.jpg

4511 data, test complete!


In [11]:
correct_dict

{'산': 193,
 '시장': 149,
 '수영장': 168,
 '놀이공원': 127,
 '미술관': 168,
 '클럽': 163,
 '호텔': 196,
 '박물관': 163,
 '동물원': 183,
 '아이스링크': 176,
 '지하철역': 173,
 '폭포': 198,
 '놀이터': 149,
 '아쿠아리움': 174,
 '절': 84,
 '다리': 170,
 '쇼핑몰': 84,
 '공원': 189,
 '공항': 143,
 '교회': 197,
 '궁궐': 161,
 '성당': 156,
 '볼링장': 191}

In [12]:
count = collections.Counter(label_list)
count

Counter({'산': 200,
         '시장': 200,
         '수영장': 196,
         '놀이공원': 187,
         '미술관': 200,
         '클럽': 201,
         '호텔': 200,
         '박물관': 200,
         '동물원': 200,
         '아이스링크': 200,
         '지하철역': 200,
         '폭포': 199,
         '놀이터': 175,
         '아쿠아리움': 199,
         '절': 139,
         '다리': 210,
         '쇼핑몰': 200,
         '공원': 199,
         '공항': 200,
         '교회': 201,
         '궁궐': 200,
         '성당': 205,
         '볼링장': 200})

In [13]:
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 4)

accuracy_dict

{'산': 0.965,
 '시장': 0.745,
 '수영장': 0.8571,
 '놀이공원': 0.6791,
 '미술관': 0.84,
 '클럽': 0.8109,
 '호텔': 0.98,
 '박물관': 0.815,
 '동물원': 0.915,
 '아이스링크': 0.88,
 '지하철역': 0.865,
 '폭포': 0.995,
 '놀이터': 0.8514,
 '아쿠아리움': 0.8744,
 '절': 0.6043,
 '다리': 0.8095,
 '쇼핑몰': 0.42,
 '공원': 0.9497,
 '공항': 0.715,
 '교회': 0.9801,
 '궁궐': 0.805,
 '성당': 0.761,
 '볼링장': 0.955}

In [14]:
accuracy = round(correct/n, 4)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8324
